In [13]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # for visualiation


gunsales_pop = pd.read_csv('./data/gunsales_population.csv')
mass_shooting = pd.read_csv('./data/Mass_shooting_cleaned.csv')
mass_shooting.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,# Casualties
0,"December 31, 2013",New York,Brooklyn,60 Glenmore Ave,0,6,6
1,"December 28, 2013",Alabama,Montgomery,954 Highland Ave,3,5,8
2,"December 26, 2013",Louisiana,Slidell,2144 First St,2,6,8
3,"December 26, 2013",Louisiana,Lockport,313 Tenth St,3,3,6
4,"December 25, 2013",New Jersey,Irvington,Nye Avenue and 21st Street,3,2,5


In [3]:
gunsales_pop.head()
gunsales_pop.state

0       United States
1       United States
2       United States
3       United States
4       United States
5       United States
6       United States
7       United States
8       United States
9       United States
10      United States
11      United States
12            Alabama
13            Alabama
14            Alabama
15            Alabama
16            Alabama
17            Alabama
18            Alabama
19            Alabama
20            Alabama
21            Alabama
22            Alabama
23            Alabama
24             Alaska
25             Alaska
26             Alaska
27             Alaska
28             Alaska
29             Alaska
            ...      
3090    West Virginia
3091    West Virginia
3092    West Virginia
3093    West Virginia
3094    West Virginia
3095    West Virginia
3096        Wisconsin
3097        Wisconsin
3098        Wisconsin
3099        Wisconsin
3100        Wisconsin
3101        Wisconsin
3102        Wisconsin
3103        Wisconsin
3104      

In [7]:
gunsales_pop.head()
gunsales_pop['State'] = gunsales_pop.state
gunsales_pop.head()

,Unnamed: 0,year,state,population,admin,handgun,long_gun,month,multiple,other,permit,totals,State
0,0,2013,United States,316234505.0,267.0,881038.0,788328.0,1,31029.0,26137.0,693076.0,2483230.0,United States
1,1,2013,United States,316234505.0,375.0,770872.0,703223.0,2,31163.0,22709.0,664252.0,2298561.0,United States
2,2,2013,United States,316234505.0,376.0,683266.0,700873.0,3,23878.0,23817.0,695386.0,2197116.0,United States
3,3,2013,United States,316234505.0,665.0,533674.0,554003.0,4,18624.0,21136.0,517224.0,1702455.0,United States
4,4,2013,United States,316234505.0,619.0,445522.0,436674.0,5,16057.0,15605.0,449993.0,1424450.0,United States


In [11]:
mass_shooting.head()
mass_shooting['year'] = str(mass_shooting['Incident Date']).split('')

In [12]:
# combined dengue_features_train data with the dengue_labels_train on year, weekofyear, and city
combined_data = pd.merge(gunsales_pop, mass_shooting, how='left', on=['year', 'State'])

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat